In [1]:
def std(m=2 ** 32, a=1103515245):
    std.current = (a * std.current) % m
    yield std.current
std.current = 1

In [2]:
import random
def generate_species(p):
    p_sum = 0
    # gen_num = next(std())
    gen_num = random.random()
    for k, v in p.items():
        p_sum += v
        if gen_num < p_sum:
            yield k
            

In [3]:
M = 0 # math expectation
right = 1
left = 0
p = [.8, .1, .03, .03, .02, .02]
while abs(1 - M) > pow(10, -10):
    p[0] = (right + left) / 2
    p = [p[i] / sum(p) for i in range(len(p))]
    M = p[1] + 2 * p[2] + 3 * p[3] + 4 * p[4] + 5 * p[5]  # p1 + 2p2 + 3p3 + 4p4 + 5p5
    if M > 1:
        left = p[0]
    else:
        right = p[0]

In [4]:
# math expectations
p_before_critical = {0: 0.8, 1: 0.1, 2: 0.03, 3: 0.03, 4: 0.02, 5: 0.02}
p_critical = {0: 0.5348837209618423, 1: 0.2325581395190789, 2: 0.0697674418557237, 3: 0.0697674418557237, 4: 0.046511627903815786, 5: 0.046511627903815786}
p_after_critical = {0: .5, 1: .05, 2: .05, 3: .10, 4: .15, 5: .15}
pbc_math = 0.1 + 2 * 0.03 + 3 * 0.03 + 4 * 0.02 + 5 * 0.02
pac_math = 0.15 + 2 * 0.05 + 3 * 0.10 + 4 * 0.15 + 5 * 0.15

# p1 + 2p2 + 3p3 + 4p4 + 5p5 = 1
# p1 + 2p + 3p + 4p + 5p => 0.1 + 2 * 0.03 + 3 * 0.03 + 4 * 0.02 + 5 * 0.02 = 
# p1 + 2p + 3p + 4p + 5p => 0.05 + 2 * 0.1 + 3 * 0.2 + 4 * 0.3 + 5 * 0.3 = 

In [5]:
# variation
# p1 + 2p2 + 3p3 ^ 2 + 4p4 ^ 3 + 5p5 ^ 4        1st
# 2p2 + 6p3 + 12p4 + 20p5        2nd
# variation = 2nd_deriative + 1st_deriative - 1st_deriative^2

pbc_variation = (2 * p_before_critical[2] + 6 * p_before_critical[3] + 12 * p_before_critical[4] + p_before_critical[5])  + pbc_math - pow(pbc_math, 2)
pac_variation = (2 * p_after_critical[2] + 6 * p_after_critical[3] + 12 * p_after_critical[4] + p_after_critical[5])  + pac_math - pow(pac_math, 2)
crit_variation = (2 * p_critical[2] + 6 * p_critical[3] + 12 * p_critical[4] + p_critical[5]) + M - pow(M, 2)

In [6]:
def death(p):
    q_prev = 0
    Q = p[0] + q_prev * p[1] + pow(q_prev, 2) * p[2] + pow(q_prev, 3) * p[3] + pow(q_prev, 4) * p[4] + pow(q_prev, 5) * p[5]
    while abs(Q - q_prev) > pow(10, -10):
        q_prev = Q
        Q = p[0] + q_prev * p[1] + pow(q_prev, 2) * p[2] + pow(q_prev, 3) * p[3] + pow(q_prev, 4) * p[4] + pow(q_prev, 5) * p[5]
    return Q
        
pbc_death = death(p_before_critical)
pac_death = death(p_after_critical)
crit_death = death(p_critical)
        

In [7]:
def simulation(p_current, p_new, n):
    k = 1  # amount of species
    for _ in range(n):
        sum_k = 0
        for _ in range(k):
            next_k = next(generate_species(p_current))
            sum_k += next_k
            p_new[next_k] += 1
        if sum_k == 0:
            break
        k = sum_k


In [ ]:
n_exp = 1000
n_iter = 20
p_new_after_crit = [0] * 6
p_new_before_crit = [0] * 6
p_new_crit = [0] * 6
for _ in range(n_exp):
    simulation(p_before_critical, p_new_before_crit, n_iter)
    simulation(p_after_critical, p_new_after_crit, n_iter)
    simulation(p_critical, p_new_crit, n_iter)
    

p_dev_after_crit = [x / sum(p_new_after_crit) for x in p_new_after_crit]
p_dev_before_crit = [x / sum(p_new_before_crit) for x in p_new_before_crit]
p_dev_crit = [x / sum(p_new_crit) for x in p_new_crit]


In [ ]:
p_dif_after_crit = [p_dev_after_crit[x] / p_after_critical[x] for x in range(len(p_dev_after_crit))]
p_dif_before_crit = [p_dev_before_crit[x] / p_before_critical[x] for x in range(len(p_dev_before_crit))]
p_dif_crit = [p_dev_crit[x] / p_critical[x] for x in range(len(p_dev_crit))]
